In [1]:
import string
import re
from pickle import load,dump
from unicodedata import normalize
from numpy import array
from numpy.random import rand
from numpy.random import shuffle
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm
import spacy

nlp = spacy.load('en_core_web_lg')
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('fr.csv')

In [3]:
df.sample()

,Unnamed: 0,eng,fr,info
107480,107480,You can't live without water.,On ne peut pas vivre sans eau.,CC-BY 2.0 (France) Attribution: tatoeba.org #466971 (CK) & #1347659 (sacredceltic)


In [4]:
tqdm.pandas()
for c in ['eng', 'fr']:
    df.loc[:, str(c+'_clean')] = df.loc[:, c].str.replace('/', ' ')
    df.loc[:, str(c+'_clean')] = df.loc[:,str(c+'_clean')].str.replace('[ ][a-z][.]', ' ')
    df.loc[:, str(c+'_clean')] = df.loc[:, str(c+'_clean')].str.replace(r'[^\w\s]', ' ')
    df.loc[:, str(c+'_clean')] = df.loc[:, str(c+'_clean')].str.replace('\d+', ' ')
    df = df.astype(str)
    df[str(c+'_clean')] = df[str(c+'_clean')].str.lower()

C:\Users\Weronika Gramacka\AppData\Local\Temp\ipykernel_16744\3677572769.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, str(c+'_clean')] = df.loc[:,str(c+'_clean')].str.replace('[ ][a-z][.]', ' ')
C:\Users\Weronika Gramacka\AppData\Local\Temp\ipykernel_16744\3677572769.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, str(c+'_clean')] = df.loc[:, str(c+'_clean')].str.replace(r'[^\w\s]', ' ')
C:\Users\Weronika Gramacka\AppData\Local\Temp\ipykernel_16744\3677572769.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, str(c+'_clean')] = df.loc[:, str(c+'_clean')].str.replace('\d+', ' ')
C:\Users\Weronika Gramacka\AppData\Local\Temp\ipykernel_16744\3677572769.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, str(c+'_clean')] = df.loc[:,str(c+'_cl

In [5]:
df.sample(3)

,Unnamed: 0,eng,fr,info,eng_clean,fr_clean
31520,31520,Rats breed rapidly.,Les rats se reproduisent rapidement.,CC-BY 2.0 (France) Attribution: tatoeba.org #35794 (CM) & #9252907 (Micsmithel),rats breed rapidly,les rats se reproduisent rapidement
71569,71569,I have a package for you.,J'ai un paquet pour vous.,CC-BY 2.0 (France) Attribution: tatoeba.org #2358771 (CK) & #8031082 (Aiji),i have a package for you,j ai un paquet pour vous
144720,144720,A monkey is climbing up a tall tree.,Un singe grimpe sur un grand arbre.,CC-BY 2.0 (France) Attribution: tatoeba.org #25874 (CM) & #138696 (hortusdei),a monkey is climbing up a tall tree,un singe grimpe sur un grand arbre


In [6]:
df['eng_len'] = df['eng_clean'].str.count(' ') + 1
df['fr_len'] = df['fr_clean'].str.count(' ') + 1

### Maxymalna dlugosc zdania 15 słów

In [7]:
df = df.drop(df[df.eng_len > 15].index)
df = df.drop(df[df.fr_len > 15].index)


In [8]:
df.sample()

,Unnamed: 0,eng,fr,info,eng_clean,fr_clean,eng_len,fr_len
26704,26704,Tom is perspiring.,Tom transpire.,CC-BY 2.0 (France) Attribution: tatoeba.org #2236813 (CK) & #5411097 (Aiji),tom is perspiring,tom transpire,4,3


In [9]:
df.shape

(191296, 8)

In [10]:
 from sklearn.utils import shuffle
df = shuffle(df)

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df,  test_size=0.1, random_state=42)
df_val, df_test = train_test_split(df_test,  test_size=0.5, random_state=42)

In [12]:
eng = df['eng_clean']
fr= df['fr_clean']

In [13]:
eng_train = df_train['eng_clean']
eng_val = df_val['eng_clean']
eng_test = df_test['eng_clean']
fr_train= df_train['fr_clean']
fr_val= df_val['fr_clean']
fr_test= df_test['fr_clean']

In [14]:
tokenizer_eng = Tokenizer()
tokenizer_fr = Tokenizer()
tokenizer_eng.fit_on_texts(eng)
tokenizer_fr.fit_on_texts(fr)

In [15]:
tokenizer_eng.word_index

{'i': 1,
 'you': 2,
 'to': 3,
 'the': 4,
 'a': 5,
 't': 6,
 'tom': 7,
 'is': 8,
 'that': 9,
 'it': 10,
 's': 11,
 'he': 12,
 'do': 13,
 'of': 14,
 'this': 15,
 'me': 16,
 'we': 17,
 'in': 18,
 'don': 19,
 'have': 20,
 'was': 21,
 'what': 22,
 'my': 23,
 'm': 24,
 'for': 25,
 'are': 26,
 'your': 27,
 'can': 28,
 're': 29,
 'be': 30,
 'want': 31,
 'she': 32,
 'not': 33,
 'know': 34,
 'like': 35,
 'on': 36,
 'with': 37,
 'they': 38,
 'did': 39,
 'how': 40,
 'all': 41,
 'at': 42,
 'his': 43,
 'll': 44,
 'go': 45,
 'think': 46,
 've': 47,
 'there': 48,
 'here': 49,
 'him': 50,
 'about': 51,
 'get': 52,
 'very': 53,
 'time': 54,
 'didn': 55,
 'and': 56,
 'were': 57,
 'no': 58,
 'one': 59,
 'why': 60,
 'will': 61,
 'going': 62,
 'up': 63,
 'out': 64,
 'her': 65,
 'as': 66,
 'had': 67,
 'has': 68,
 'just': 69,
 'need': 70,
 'd': 71,
 'mary': 72,
 'who': 73,
 'if': 74,
 'good': 75,
 'let': 76,
 'would': 77,
 'so': 78,
 'an': 79,
 'should': 80,
 'see': 81,
 'come': 82,
 'tell': 83,
 'where': 84,

In [16]:
eng_vocab_size = len(tokenizer_eng.word_index) + 1
fr_vocab_size = len(tokenizer_fr.word_index) + 1

In [17]:
eng_vocab_size

14650

In [18]:
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [19]:
eng_max = max(len(text.split(' ')) for text in eng)
eng_max

15

In [20]:
eng_train = encode_sequences(tokenizer_eng, 15, eng_train)
eng_val = encode_sequences(tokenizer_eng, 15, eng_val)
eng_test = encode_sequences(tokenizer_eng, 15, eng_test)

fr_train = encode_sequences(tokenizer_fr, 15, fr_train)
fr_val = encode_sequences(tokenizer_fr, 15, fr_val)
fr_test = encode_sequences(tokenizer_fr, 15, fr_test)

In [21]:
def create_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units//2))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units//4))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units//8, return_sequences=True))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(out_vocab, activation='softmax'))
    return model

In [22]:
tf.keras.backend.clear_session()
model = create_model(eng_vocab_size, fr_vocab_size, 15, 15, 1024)
opt = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy')

C:\Users\Weronika Gramacka\anaconda3\envs\python\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 1024)          15001600  
                                                                 
 lstm (LSTM)                 (None, 1024)              8392704   
                                                                 
 repeat_vector (RepeatVector  (None, 15, 1024)         0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 512)               3147776   
                                                                 
 repeat_vector_1 (RepeatVect  (None, 15, 512)          0         
 or)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 256)               7

In [ ]:
filename = 'model_GUM_2_2.h1'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history = model.fit(eng_train, fr_train.reshape(fr_train.shape[0], fr_train.shape[1], 1), epochs=30, batch_size=256, validation_data=(eng_val, fr_val.reshape(fr_val.shape[0], fr_val.shape[1], 1)),   callbacks=[checkpoint], verbose=1)

Epoch 1/30
349/673 [==============>...............] - ETA: 2:55 - loss: 3.4390

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

In [26]:
model = load_model('model_GUM_2_2.h1')

In [27]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [52]:
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [54]:
def evaluate_model(model, tokenizer, test, test_t):
    actual, predicted, bleu = list(), list(), list()
    for i, x in enumerate(test):
        x = x.reshape((1, x.shape[0]))
        translation = predict_sequence(model, tokenizer, x)
        predicted.append(translation.split())
    return predicted

In [55]:
eng_test[0]

array([ 22,  13,   2, 142, 371, 446,  14,   0,   0,   0,   0,   0,   0,
         0,   0])

In [56]:
df_test.head(5)

,Unnamed: 0,eng,fr,info,eng_clean,fr_clean,eng_len,fr_len
120892,120892,What do you feel most proud of?,De quoi êtes-vous la plus fière ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6861814 (CK) & #6861903 (GB3),what do you feel most proud of,de quoi êtes vous la plus fière,8,8
150631,150631,I know what you're trying to tell me.,Je sais ce que tu essaies de me dire.,CC-BY 2.0 (France) Attribution: tatoeba.org #2376410 (CK) & #8110625 (Aiji),i know what you re trying to tell me,je sais ce que tu essaies de me dire,10,10
22970,22970,Do you smell that?,Sens-tu cela ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1680469 (CM) & #4598366 (sacredceltic),do you smell that,sens tu cela,5,5
186842,186842,He was kind enough to show me the way to the station.,Il fut assez aimable de me montrer le chemin jusqu'à la station.,CC-BY 2.0 (France) Attribution: tatoeba.org #300031 (CK) & #132697 (Micsmithel),he was kind enough to show me the way to the station,il fut assez aimable de me montrer le chemin jusqu à la station,13,14
73018,73018,I'm willing to apologize.,Je veux faire mes excuses.,CC-BY 2.0 (France) Attribution: tatoeba.org #1841621 (CK) & #1843059 (sacredceltic),i m willing to apologize,je veux faire mes excuses,6,6


In [57]:
preds = evaluate_model(model, tokenizer_fr, eng_test, fr_test)

In [71]:
type(preds[0])

list

In [ ]:
score = sentence_bleu(reference, candidate)

In [75]:
def bleu_score(orginal, translated):
    orginal = [orginal.split(' ')]
    translated = translated.split(' ')
    score = sentence_bleu(orginal, translated)
    return score

In [77]:
df_test['fr_preds'] = preds
df_test['fr_preds']=df_test.fr_preds.apply(lambda x: ' '.join(x))
df_test['bleu_score']=df_test.apply(lambda x: bleu_score(x.fr_clean, x.fr_preds), axis=1)

C:\Users\Weronika Gramacka\anaconda3\envs\python\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Weronika Gramacka\anaconda3\envs\python\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Weronika Gramacka\anaconda3\envs\python\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

In [78]:
df_test.head(5)

,Unnamed: 0,eng,fr,info,eng_clean,fr_clean,eng_len,fr_len,fr_preds,bleu_score
120892,120892,What do you feel most proud of?,De quoi êtes-vous la plus fière ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6861814 (CK) & #6861903 (GB3),what do you feel most proud of,de quoi êtes vous la plus fière,8,8,de quoi es vous un temps,6.010456e-155
150631,150631,I know what you're trying to tell me.,Je sais ce que tu essaies de me dire.,CC-BY 2.0 (France) Attribution: tatoeba.org #2376410 (CK) & #8110625 (Aiji),i know what you re trying to tell me,je sais ce que tu essaies de me dire,10,10,je sais ce que vous devrais de,2.831942e-01
22970,22970,Do you smell that?,Sens-tu cela ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1680469 (CM) & #4598366 (sacredceltic),do you smell that,sens tu cela,5,5,tu vous que tu,1.003274e-231
186842,186842,He was kind enough to show me the way to the station.,Il fut assez aimable de me montrer le chemin jusqu'à la station.,CC-BY 2.0 (France) Attribution: tatoeba.org #300031 (CK) & #132697 (Micsmithel),he was kind enough to show me the way to the station,il fut assez aimable de me montrer le chemin jusqu à la station,13,14,il était trop plus pour pour lui aller la la la,9.056778e-232
73018,73018,I'm willing to apologize.,Je veux faire mes excuses.,CC-BY 2.0 (France) Attribution: tatoeba.org #1841621 (CK) & #1843059 (sacredceltic),i m willing to apologize,je veux faire mes excuses,6,6,je suis habitué à mes mes,1.384293e-231
